This is a series of notebooks were I explore the [**fastai**](https://www.fast.ai/) [**library**](https://docs.fast.ai/) (with a focus on computer vision given the nature of the competition). More parts will be available in the upcoming weeks, so stay tuned!

In this first part, I will show you:

1. How to read an image
2. How to read a mask from a [**run-length encoding**](https://en.wikipedia.org/wiki/Run-length_encoding)
3. How to combine both steps to get a bunch of images and masks

Let's start!

# Loading an image

Alright, to get started, let's get an image from the training dataset.
For that, we will need only two things:

1. path to the image
2. [`open_image`](https://docs.fast.ai/vision.image.html#open_image) utility function

In [ ]:
from fastai.vision import open_image

In [ ]:
img_id = "0002cc93b"
img_path = f"../input/severstal-steel-defect-detection/train_images/{img_id}.jpg"

In [ ]:
open_image(img_path)

In [ ]:
# One channel from RGB but not necessary here (since all three channels are the same).
open_image(img_path, convert_mode = "L")

In [ ]:
img = open_image(img_path)

In [ ]:
# Displaying the first 5 * 5 patch
img.px[:, :5, :5]

In [ ]:
# The image shape: (channels, width, height)
img.px.shape

In [ ]:
# Notice that all channels contain the same information here.
print((img.px[0, : , :] == img.px[1, :, :]).all())
print((img.px[1, : , :] == img.px[2, :, :]).all())

That was easy!

# Loading a mask

Next, let's load a mask for the given image. As you will see, the masks are stored as textual representation (run-length encoding shortned as rle) rather than images to save space. This is a slightly more complicated task but 
nothing insurmontable. For that, we will need three things:
    
1. mapping between the image and the mask (or masks if many)
2. the mask's rle
3. [`open_mask_rle utiliy`](https://docs.fast.ai/vision.image.html#open_mask_rle) function

To get the mapping and the rle, we need to open the train CSV. Will be using pandas for that.

In [ ]:
import pandas as pd
train_df = pd.read_csv("../input/severstal-steel-defect-detection/train.csv")

In [ ]:
train_df.head()

In [ ]:
# Will extract img_id and class_id
train_df["img_id"] = train_df["ImageId_ClassId"].str.split(".").str[0]
train_df["class_id"] =  train_df["ImageId_ClassId"].str.split(".").str[1].str.split('_').str[1]

In [ ]:
train_df.head()

Let's find the rle corresponding to the `img_id` (id of the displayed image). 

In [ ]:
train_df.loc[lambda df: df["img_id"] == img_id, ["EncodedPixels", "class_id"]]

In [ ]:
# The mask is the first elemen
mask_rle = train_df.loc[lambda df: df["img_id"] == img_id, "EncodedPixels"].values[0]

Great! So we have one mask (represented using rle) with `class_id` 1 for the given image. Let's
plot the mask!

In [ ]:
from fastai.vision import open_mask_rle
mask_shape = (img.px.shape[1], img.px.shape[2])
mask = open_mask_rle(mask_rle, shape=mask_shape)
mask

Hum, that looks like a mask but wrongly shaped. What went wrong?
Well, this is a quirk of the rle format. I won't delve into too much details but
just remember that you need to rotate the mask 90 degrees (counter-clockwise).
To do so, one can use the transpose operation. 

In [ ]:
from fastai.vision import ImageSegment
# Need to create a mask using the ImageSegment class
mask = ImageSegment(mask.data.transpose(2, 1))
mask

Looks better!

# Combining both steps in a pipeline

Before doing that, let's plot both the image and the mask in a single plot.

In [ ]:
img.show(y=mask, figsize=(20, 10), title=f"{img_id} with mask, label 1")

I was planning to run a data pipeline without doing any intermediate steps. However, I was unable to do so so far. If you know how to do it, please let me know in the comments section.
For now, I have a step where I extract and save masks (one mask per image). 

Let's go through this.

In [ ]:
!mkdir ../masks

In [ ]:
import math
import torch
from fastai.vision import open_mask_rle, ImageSegment

def get_and_save_mask(img_id, df, shape=(1600, 256)):
    """ Extract the mask(s) for each image. The mask could be None."""
    # Shape: (width, height)
    # One mask (or none) per image.
    masks = []
    rle_df = df.loc[df["img_id"] == img_id, ['class_id', 'EncodedPixels']]
    # Not all images have masks
    for row in rle_df.itertuples():
        rle = row.EncodedPixels
        class_id = row.class_id
        if isinstance(rle, float) and math.isnan(rle):
            continue
        one_mask = open_mask_rle(rle, shape=shape)
        one_mask = int(class_id) * one_mask.data
        masks.append(one_mask)
    if len(masks) == 0:
        return
    stacked_mask = torch.stack(masks, dim=0).sum(dim=0)
    mask_img = ImageSegment(stacked_mask.reshape((1, shape[0], shape[1])).transpose(2, 1))
    mask_img.save(f"../masks/{img_id}.jpg")

In [ ]:
# Run over all the train images
for img_id in train_df["img_id"].unique():
    get_and_save_mask(img_id, train_df)

Let's load one of these saved masks. 

In [ ]:
from fastai.vision import open_mask
open_mask("../masks/0025bde0c.jpg")

Awesome, it worked! Alright, now that we have both images and masks saved 
as files, let's load a bunch of them.

Before we dive deep, here is the plan to get the data pipeline: 

* contruct a DataFrame that contains only images having at least one mask (this will be called
`with_masks_df`)
* Load images using the `from_df` method
* Extract labels using the `label_from_func` method: this takes a function that reads
one image input path and outputs a mask path. Since we have multiple classes (0 for background
and 1 through 4 for the different defect types), these should be passed as well.
* Transform the images and masks. Notice that this step is optional but I am 
adding it to get resized images (since I pass the `size` variable)
* Finally, use the `databunch` method to create a [**DataBunch**](https://docs.fast.ai/basic_data.html#DataBunch). Again, this isn't necessary but will comes handy in the next notebook so I want you to get used to the concept.



In [ ]:
# Filtering images with at least one mask
with_masks_df = (train_df.groupby('img_id')['EncodedPixels'].count() 
                      .reset_index()
                      .rename(columns={"EncodedPixels": "n_masks"}))
with_masks_df = with_masks_df.loc[lambda df: df["n_masks"] > 0, :]

In [ ]:
# data pipeline
from fastai.vision import SegmentationItemList, get_transforms

train_folder = "../input/severstal-steel-defect-detection/train_images/"
sl = SegmentationItemList.from_df(with_masks_df, train_folder, suffix=".jpg")
size = 256
batch_size = 16
data = (sl.split_none()
          .label_from_func(lambda x : str(x).replace(train_folder, '../masks/'),
                           classes=[0, 1, 2, 3, 4])
          .transform(get_transforms(), size=size, tfm_y=True)
          .databunch(bs=batch_size))

Now that the pipeline is defined, we can visualize some images with the associated masks.

In [ ]:
data.show_batch()

That's it for this first notebook. In the next one, we will be building upon these foundations
and train a [**U-net**](https://arxiv.org/abs/1505.04597) model. 
Stay tuned. ;)